In [2]:
%reload_ext autoreload
%autoreload 2

- https://ax.dev/tutorials/gpei_hartmann_service.html

In [3]:
from ax import optimize

In [4]:
from reprpo.training import train
import tyro
from reprpo.experiments import experiment_configs
from reprpo.interventions import Interventions, DPOConfig, ReprPOConfig
from reprpo.interventions.losses import Losses
from reprpo.interventions.transforms import Transforms

# training_args = tyro.extras.overridable_config_cli(experiment_configs)
# training_args



In [5]:
tuner_kwargs = dict(
    verbose=0,
    base_model= 'TinyLlama/TinyLlama-1.1B-Chat-v1.0', # ideally would be SFT
    batch_size=76,
    load_in_4bit=True,
    collection_layers_side = [8, 10, 12, 14, 16, 18],
    eval_samples=24,
)

def override(cfg, overrides):
    for k, v in overrides.items():
        if hasattr(cfg, k):
            setattr(cfg, k, v)
        else:
            print(f"WARNING: {k} not found in config")
    return cfg

def objective_func(**kwargs):
    cfg = experiment_configs['side-ether-prefvec'][1]
    override(cfg, kwargs)
    override(cfg, tuner_kwargs)
    print(cfg)
    return train(cfg)

In [ ]:
cfg._

In [10]:
cfg=experiment_configs['side-ether-prefvec'][1]
cfg.loss.eps
import operator
operator.attrgetter('loss.eps')(cfg)
operator.attrsetter

AttributeError: module 'operator' has no attribute 'attrsetter'

In [ ]:
# r = objective_func()
# r.valuue

note you can have dependant params
- https://github.com/facebook/Ax/issues/1454

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="ax")

In [13]:
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.utils.measurement.synthetic_functions import branin


ax_client = AxClient(verbose_logging=False)
ax_client.create_experiment(
    name="branin_test_experiment",
    parameters=[
         {"name": "lr", "type": "range", "bounds": [1e-6, 0.4], "log_scale": True},
        {"name": "β", "type": "range", "bounds": [1e-6, 0.4], "log_scale": True},
        {
            "name": "use_dpo_loss",
            "type": "choice",
            "values": [False, True],
        },
        {
            "name": "use_nll_loss",
            "type": "choice",
            "values": [False, True],
        },
        {
            "name": "weight_tokens",
            "type": "choice",
            "values": [False, True],
        },
        {
            "name": "use_orth_loss",
            "type": "choice",
            "values": [False, True],
        },
    ],
    objectives={"objective_func": ObjectiveProperties(minimize=False)},
)

for _ in range(15):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=objective_func(**parameters))

best_parameters, metrics = ax_client.get_best_parameters()

[INFO 09-24 13:47:28] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 09-24 13:47:28] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lr. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-24 13:47:28] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter β. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-24 13:47:28] ax.service.utils.instantiation: Inferred value type of ParameterType.BOOL for parameter use_dpo_loss. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
/workspace/repr-preferenc

ReprPOConfig(dataset='us_history_textbook', verbose=1, dev=False, load_in_4bit=True, load_in_8bit=False, use_gradient_checkpointing=False, batch_size=76, n_samples=1800, eval_samples=24, max_length=196, max_prompt_length=96, base_model='TinyLlama/TinyLlama-1.1B-Chat-v1.0', lr=0.00019195269673638238, collection_layers_side=[8, 10, 12, 14, 16, 18], collection_keys_in=('base_model.model.model.layers.{layer}.self_attn.o_proj', 'base_model.model.model.layers.{layer}.mlp.down_proj'), collection_keys_out=('base_model.model.model.layers.{layer}.self_attn.q_proj', 'base_model.model.model.layers.{layer}.self_attn.k_proj', 'base_model.model.model.layers.{layer}.self_attn.v_proj', 'base_model.model.model.layers.{layer}.mlp.gate_proj', 'base_model.model.model.layers.{layer}.mlp.up_proj'), collect_input=True, loss=PrefVecLossConfig(eps=1e-12, β=0.2, use_orth_loss=True, use_angle_loss=False, use_dpo_loss=True, use_nll_loss=True, weight_tokens=False), transform=ETHERConfig(nb=4, Htype='etherplus', eth

epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-AdamW8bit,▁▇███▇▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁▁
res/acc/oos,▁
res/acc/rnd,▁
res/acc/test,▁
res/acc/train,▁
res/coherency[cho-rej]/oos,▁
res/coherency[cho-rej]/rnd,▁
res/coherency[cho-rej]/test,▁
res/coherency[cho-rej]/train,▁
res/coherency[pi-base]/oos,▁


trainable params: 9011200 || all params: 624617472 || trainable%: 1.4426749817206521
|INFO| Prompts truncated 66.83%
|INFO| Chosens truncated 24.33%
|INFO| WANDB url = https://wandb.ai/wassname/reprpo2/runs/v0fv7u3e
|INFO| max optimiser steps 23
|INFO| accumulate_grad_batches 1
|INFO| accumulated batch size 76
|INFO| epochs 1


Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/workspace/repr-preference-optimization/.venv/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]


Generated on batch 4
|INFO| --------------------------------------------------------------------------------
|INFO| **Adapter:`side-ETHER-PrefVec_us_history_textbook` generation**`
|INFO| `Q1: (100 words): I would prefer to live in the "The Polity" by Neal Asher, a society that values individual`
|INFO| --------------------------------------------------------------------------------
|INFO| **Adapter:`base` generation**`
|INFO| `Q1: (100 words): I would prefer to live in the Society of Permutation City by Greg Egan.

Why: Perm`
|INFO| ================================================================================

Generated on batch 9
|INFO| --------------------------------------------------------------------------------
|INFO| **Adapter:`side-ETHER-PrefVec_us_history_textbook` generation**`
|INFO| `Q1: (100 words): I would prefer to live in the "The Polity" by Neal Asher, a science fiction society that`
|INFO| --------------------------------------------------------------------------

In [ ]:
best_parameters, best_values, experiment, model = optimize(
    #  experiment_name="test",
    # objective_name="hartmann6",
        parameters=[
          {
            "name": "lr",
            "type": "range",
            "bounds": [-10.0, 10.0],
          },
          {
            "name": "x2",
            "type": "range",
            "bounds": [-10.0, 10.0],
          },
        ],
        # returns acc improvement on oos
        evaluation_function=objective_func,
        minimize=False,
        total_trials=30,  # Optional.
    )
best_parameters